In [1]:
'''
This Python file will read each of the clean annual data csv files and create 6 new split data files so that the
different climate tables can be loaded into a relational database. 
'''
import pandas as pd
import os
import glob
import warnings  
# Settings the warnings to be ignored
warnings.filterwarnings('ignore')

dir_path = r'resources'
all_files = glob.glob(os.path.join(dir_path , "*.csv"), ) 

In [2]:
file_list = []
for filename in all_files:
    all_data_df = pd.read_csv(filename, index_col=None, header=0, on_bad_lines='warn', engine='python')
    file_list.append(all_data_df)

#create one large DataFrame of all years of the data    
entire_df = pd.concat(file_list, ignore_index=True)

In [3]:
# what datatypes do I have? What is the DATE field?
entire_df.dtypes

DATE                 object
WND_direction         int64
WND_dQC              object
WND_type             object
WND_speed             int64
WND_speedQC          object
SKY_ceiling           int64
SKY_ceilingQC         int64
SKY_ceilingDC        object
SKY_CAVOK            object
VIS_distance          int64
VIS_distanceQC       object
VIS_variability      object
VIS_variabilityQC    object
AIR_temp              int64
AIR_tempQC           object
DEW_temp              int64
DEW_tempQC           object
SLP_pressure          int64
SLP_pressureQC       object
dtype: object

In [4]:
# dimensions of the DataFrame before duplicate
entire_df.shape

(56147, 20)

In [5]:
# there are some duplicate rows that give errors when trying to use this data.
duplicateRows_DF = entire_df[entire_df.duplicated()]
duplicateRows_DF_count = entire_df[entire_df.duplicated()].count()
print("Duplicate Rows", duplicateRows_DF, sep='\n')

Duplicate Rows
                      DATE  WND_direction WND_dQC WND_type  WND_speed  \
22501  2016-11-22 07:54:00            100       5        N         46   
23669  2017-01-28 11:54:00            100       5        N         57   
32705  2018-11-02 12:36:00             70       5        N         46   
32755  2018-11-08 22:57:00             80       5        N         72   
32758  2018-11-08 23:58:00             80       5        N         46   
34700  2019-02-19 02:05:00             40       5        N        108   
35168  2019-03-16 00:10:00             90       5        N         51   
35387  2019-03-30 16:57:00             70       5        N         98   
35755  2019-04-29 20:56:00            110       5        N         26   
37262  2019-08-19 06:56:00            170       5        N         31   
38014  2019-09-29 19:57:00            340       5        N         31   
40627  2020-02-14 06:59:00             60       5        N        118   
40884  2020-03-02 09:58:00          

In [6]:
# DATE field is a string. converted to datetime
entire_df['DATE']= pd.to_datetime(entire_df['DATE']) 
entire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56147 entries, 0 to 56146
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   DATE               56147 non-null  datetime64[ns]
 1   WND_direction      56147 non-null  int64         
 2   WND_dQC            56147 non-null  object        
 3   WND_type           56147 non-null  object        
 4   WND_speed          56147 non-null  int64         
 5   WND_speedQC        56147 non-null  object        
 6   SKY_ceiling        56147 non-null  int64         
 7   SKY_ceilingQC      56147 non-null  int64         
 8   SKY_ceilingDC      56147 non-null  object        
 9   SKY_CAVOK          56147 non-null  object        
 10  VIS_distance       56147 non-null  int64         
 11  VIS_distanceQC     56147 non-null  object        
 12  VIS_variability    56147 non-null  object        
 13  VIS_variabilityQC  56147 non-null  object        
 14  AIR_te

In [7]:
# see if it will drop the duplicate rows now... some 2023 duplicate rows still present
entire_df.drop_duplicates(
    subset=None, 
    inplace=True, 
    ignore_index=False)
entire_df.shape

(56101, 20)

In [8]:
# drop rows that have duplicated timestamp values even if other data is not duplicated. 
entire_df.drop_duplicates(
    subset=['DATE'],     
    inplace=True, 
)
entire_df.shape

(56062, 20)

In [11]:
# Write out entire data to csv format
if os.path.exists('resources/all_data.csv'):
    os.remove('resources/all_data.csv')
    
entire_df.to_csv('resources/all_data.csv', index=False)

In [12]:
# create new DataFrames for each of the 6 climate variables to load into their own database tables
# Start with the wind variables
wnd_df = entire_df.filter(['DATE','WND_direction', 'WND_dQC', 'WND_type', 'WND_speed', 'WND_speedQC'])
wnd_df.tail(100)

,DATE,WND_direction,WND_dQC,WND_type,WND_speed,WND_speedQC
56047,2023-04-07 00:11:00,90,5,N,46,5
56048,2023-04-07 00:54:00,90,5,N,51,5
56049,2023-04-07 01:54:00,100,5,N,51,5
56050,2023-04-07 02:37:00,80,5,N,67,5
56051,2023-04-07 02:52:00,100,5,N,67,5
...,...,...,...,...,...,...
56142,2023-04-15 04:54:00,80,5,N,77,5
56143,2023-04-15 11:54:00,60,5,N,57,5
56144,2023-04-15 12:54:00,60,5,N,57,5
56145,2023-04-15 15:54:00,70,5,N,62,5


In [13]:
# Write out wind data to csv format
if os.path.exists('resources/WND/wnd_data.csv'):
    os.remove('resources/WND/wnd_data.csv')
wnd_df.to_csv('resources/WND/wnd_data.csv', index=False)

In [14]:
# Create the SKy conditions DataFrame
sky_df = entire_df.filter(['DATE','SKY_ceiling', 'SKY_ceilingQC', 'SKY_ceilingDC', 'SKY_CAVOK'])
sky_df.tail(20)

,DATE,SKY_ceiling,SKY_ceilingQC,SKY_ceilingDC,SKY_CAVOK
56127,2023-04-12 12:26:00,671,5,M,N
56128,2023-04-12 14:54:00,1433,5,M,N
56129,2023-04-12 18:54:00,1829,5,M,N
56130,2023-04-13 14:54:00,1341,5,M,N
56131,2023-04-13 15:54:00,1829,5,M,N
56132,2023-04-13 16:54:00,2134,5,M,N
56133,2023-04-14 13:54:00,1341,5,M,N
56134,2023-04-14 14:54:00,1341,5,M,N
56135,2023-04-14 16:54:00,22000,5,C,N
56136,2023-04-14 19:54:00,1494,5,M,N


In [15]:
# write out sky into csv file
if os.path.exists('resources/SKY/sky_data.csv'):
    os.remove('resources/SKY/sky_data.csv')
sky_df.to_csv('resources/SKY/sky_data.csv', index=False)

In [16]:
# Create visibility DataFrame
vis_df = entire_df.filter(['DATE','VIS_distance', 'VIS_distanceQC', 'VIS_variability', 'VIS_variabilityQC'])
vis_df.head(20)

,DATE,VIS_distance,VIS_distanceQC,VIS_variability,VIS_variabilityQC
0,2013-01-01 00:10:00,4828,5,N,5
1,2013-01-01 00:54:00,6437,5,N,5
2,2013-01-01 01:32:00,6437,5,N,5
3,2013-01-01 02:43:00,4828,5,N,5
4,2013-01-01 02:54:00,16093,5,N,5
5,2013-01-01 03:25:00,16093,5,N,5
6,2013-01-01 03:40:00,16093,5,N,5
7,2013-01-01 04:49:00,16093,5,N,5
8,2013-01-01 04:54:00,16093,5,N,5
9,2013-01-01 06:51:00,16093,5,N,5


In [17]:
# write out visibility data into csv file
if os.path.exists('resources/VIS/vis_data.csv'):
    os.remove('resources/VIS/vis_data.csv')
vis_df.to_csv('resources/VIS/vis_data.csv', index=False)

In [18]:
# Create air temp DataFrame
air_temp_df = entire_df.filter(['DATE','AIR_temp', 'AIR_tempQC'])
air_temp_df.head(20)

,DATE,AIR_temp,AIR_tempQC
0,2013-01-01 00:10:00,280,5
1,2013-01-01 00:54:00,278,5
2,2013-01-01 01:32:00,280,5
3,2013-01-01 02:43:00,280,5
4,2013-01-01 02:54:00,283,5
5,2013-01-01 03:25:00,290,5
6,2013-01-01 03:40:00,290,5
7,2013-01-01 04:49:00,290,5
8,2013-01-01 04:54:00,289,5
9,2013-01-01 06:51:00,290,5


In [19]:
# write out air temp data into csv file
if os.path.exists('resources/TMP/temp_data.csv'):
    os.remove('resources/TMP/temp_data.csv')
air_temp_df.to_csv('resources/TMP/temp_data.csv', index=False)

In [20]:
# Create dew point DataFrame
dew_point_df = entire_df.filter(['DATE','DEW_temp', 'DEW_tempQC'])
dew_point_df.tail()

,DATE,DEW_temp,DEW_tempQC
56142,2023-04-15 04:54:00,230,7
56143,2023-04-15 11:54:00,230,7
56144,2023-04-15 12:54:00,230,7
56145,2023-04-15 15:54:00,222,5
56146,2023-04-15 23:54:00,240,7


In [21]:
# write out dew point data into csv file
if os.path.exists('resources/DEW/dew_data.csv'):
    os.remove('resources/DEW/dew_data.csv')
dew_point_df.to_csv('resources/DEW/dew_point.csv', index=False)

In [22]:
# Create sea level pressure DataFrame
sea_level_pressure_df = entire_df.filter(['DATE','SLP_pressure', 'SLP_pressureQC'])
sea_level_pressure_df.head(25)

,DATE,SLP_pressure,SLP_pressureQC
0,2013-01-01 00:10:00,260,5
1,2013-01-01 00:54:00,261,5
2,2013-01-01 01:32:00,260,5
3,2013-01-01 02:43:00,260,5
4,2013-01-01 02:54:00,261,5
5,2013-01-01 03:25:00,260,5
6,2013-01-01 03:40:00,260,5
7,2013-01-01 04:49:00,260,5
8,2013-01-01 04:54:00,256,5
9,2013-01-01 06:51:00,250,5


In [23]:
# write out sea level data into csv file
if os.path.exists('resources/SLP/sea_level_pressure.csv'):
    os.remove('resources/SLP/sea_level_pressure.csv')
sea_level_pressure_df.to_csv('resources/SLP/sea_level_pressure.csv', index=False)

In [24]:
print('all done!')

all done!
